Datenbank vorbereiten
 - Indexierung erstellen

In [ ]:
import sqlite3

conn = sqlite3.connect('data/SOLKAT_DACH.gpkg')
cursor = conn.cursor()

index_name = "GWR_EGID_INDEX"

create_index_sql = "CREATE INDEX {0} ON SOLKAT_CH_DACH(GWR_EGID)".format(index_name)

cursor.execute(create_index_sql)

conn.commit()
conn.close()

Daten einlesen aus Building Datenbank

In [ ]:
import sqlite3
import numpy as np
import pandas as pd

conn = sqlite3.connect('data/data.sqlite')                                
cursor = conn.cursor()

# query building DB
sql = "SELECT EGID, GGDENAME, GDEKT, GKODE, GKODN from building;"

df_building = pd.read_sql_query(sql, conn)

print(df_building)

print(df_building.head())

# Verbindung schließen
conn.close()

#df_building.to_csv("data/EGID.csv", index=False)

Daten einlesen aus Dach Datenbank

In [ ]:
import sqlite3
import numpy as np
import pandas as pd

conn = sqlite3.connect('data/SOLKAT_DACH.gpkg')
cursor = conn.cursor()

df_dach = pd.DataFrame()

# Inhalt von Dataframe löschen
df_dach.drop(index=df_dach.index, inplace=True)

# Schleife über jede Gruppe von 500 EGIDs
for i in range(0, len(df_building), 500):
    # Aktuelle Gruppe von EGIDs auswählen
    egid_group = df_building["EGID"].iloc[i:i+500]
    
    # WHERE-Klausel für die Abfrage erstellen
    where_clause = "WHERE " + " OR ".join(["GWR_EGID='{}'".format(egid) for egid in egid_group])
    
    # SQL-Abfrage ausführen
    sql =  """
        SELECT 
            SUM(FLAECHE) AS Summe_Flaeche, 
            SUM(STROMERTRAG) AS Summe_Stromertrag, 
            SUM(FLAECHE_KOLLEKTOREN) AS Summe_Flaeche_Kollektoren, 
            GWR_EGID AS EGID
        FROM 
            SOLKAT_CH_DACH
        {}
        GROUP BY
            GWR_EGID;
        """.format(where_clause)
    
    result = pd.read_sql_query(sql, conn)

    # Ergebnis dem bestehenden DataFrame "df_dach" hinzufügen
    df_dach = df_dach.append(result, ignore_index=True)


# Verbindung schließen
conn.close()


# Spalte EGID muss für merge gleicher Datentyp wie df_building/EGID haben
df_dach["EGID"] = df_dach["EGID"].astype(str)

# Mergen und Zeilen behalten, die in beiden Dataframes gefunden wurden
df_merge = pd.merge(df_building, df_dach, how='inner', on='EGID')

df_merge.to_csv("data/DachInfo.csv", index=False)
